In [1]:
import torch
from safetensors import safe_open
import torch.nn.functional as F

In [2]:
def read_tensor(path):
    tensors = {}
    with safe_open(path, framework="pt", device="cpu") as f:
        for key in f.keys():
            tensors[key] = f.get_tensor(key)
        
    return tensors["data"]

In [30]:
PATHS = [
    "/fsx/phuc/new_workspace/experiments/infini_attention_8b_llama/exp35_8b_llama_16384_ctx_length_and_8192_segment_length_and_2m_bs_and_needle_finetuning/checkpoints/663/model/model/decoder/3/pp_block/attn/model_balance_factors_pp-rank-0-of-1_tp-rank-0-of-4.safetensors",
    "/fsx/phuc/new_workspace/experiments/infini_attention_8b_llama/exp35_8b_llama_16384_ctx_length_and_8192_segment_length_and_2m_bs_and_needle_finetuning/checkpoints/1326/model/model/decoder/3/pp_block/attn/qkv_proj/model_weight_pp-rank-0-of-1_tp-rank-2-of-4.safetensors",
    "/fsx/phuc/new_workspace/experiments/infini_attention_8b_llama/exp35_8b_llama_16384_ctx_length_and_8192_segment_length_and_2m_bs_and_needle_finetuning/checkpoints/1326/model/model/decoder/3/pp_block/attn/model_balance_factors_pp-rank-0-of-1_tp-rank-2-of-4.safetensors",
    "/fsx/phuc/new_workspace/experiments/infini_attention_8b_llama/exp32_200m_infini_llama2_256_ctx_length_and_64_segment_length_and_2m_bs/checkpoints/5000/model/model/decoder/0/pp_block/attn/model_balance_factors_pp-rank-0-of-1_tp-rank-0-of-2.safetensors",
]

In [32]:
tensors

{'data': tensor([0.0156, 0.0156, 0.0156, 0.0156], dtype=torch.bfloat16)}

In [33]:
F.sigmoid(tensors["data"])

tensor([0.5039, 0.5039, 0.5039, 0.5039], dtype=torch.bfloat16)

In [12]:
tensors["data"]

NameError: name 'tensors' is not defined

In [3]:
weights = read_tensor("/fsx/phuc/projects/nanotron/checkpoints/exp33_debug/20/model/model/decoder/1/pp_block/attn/model_balance_factors_pp-rank-0-of-1_tp-rank-1-of-2.safetensors")

In [4]:
weights

tensor([-1.6764e-06,  1.6019e-06, -1.7881e-06,  1.5944e-06],
       dtype=torch.bfloat16)

In [5]:
weights * 10**6

tensor([-1.6797,  1.6016, -1.7891,  1.5938], dtype=torch.bfloat16)

In [6]:
F.sigmoid(weights)

tensor([0.5000, 0.5000, 0.5000, 0.5000], dtype=torch.bfloat16)

In [8]:
F.sigmoid(weights * 10**6)

tensor(0.3945, dtype=torch.bfloat16)

### Gradients

In [12]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

In [13]:
root_dir = '/fsx/phuc/projects/nanotron/debug/runs/exp33_debug_normalize_grad_of_balances_and_divide_by_10_and_lr_0.00015'  # replace with your actual root directory

grad_means = {}
grad_vars = {}

In [14]:
for train_step in sorted(os.listdir(root_dir)):
    train_step_dir = os.path.join(root_dir, train_step)
    if not os.path.isdir(train_step_dir):
        continue

    for grad_file in os.listdir(train_step_dir):
        if grad_file.endswith('.pt'):
            tensor_name = grad_file[:-3]  # remove the '.pt' extension
            grad_path = os.path.join(train_step_dir, grad_file)
            
            # Load the tensor
            grad_tensor = torch.load(grad_path)
            
            # Ensure tensor is on CPU and numpy
            grad_tensor_np = grad_tensor.cpu().numpy()
            
            # Calculate mean and variance
            mean = np.mean(grad_tensor_np)
            variance = np.var(grad_tensor_np)
            
            if tensor_name not in grad_means:
                grad_means[tensor_name] = []
                grad_vars[tensor_name] = []
            
            grad_means[tensor_name].append(mean)
            grad_vars[tensor_name].append(variance)

In [15]:
steps = range(len(next(iter(grad_means.values()))))  # assume same number of steps for all tensors

plt.figure(figsize=(10, 6))

for tensor_name in grad_means:
    means = grad_means[tensor_name]
    vars = grad_vars[tensor_name]
    
    lower_bound = [m - np.sqrt(v) for m, v in zip(means, vars)]
    upper_bound = [m + np.sqrt(v) for m, v in zip(means, vars)]
    
    plt.plot(steps, means, label=tensor_name)
    plt.fill_between(steps, lower_bound, upper_bound, alpha=0.3)

plt.xlabel('Training Step')
plt.ylabel('Mean of Gradients')
plt.title('Gradient Means and Variances over Training Steps')
plt.legend()
plt.grid(True)
plt.show()

StopIteration: 

In [18]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np

def load_and_plot_gradients(root_dir):
    # Dictionary to store gradients for each tensor across steps
    gradients = {}

    # Walk through the directory structure
    for step_dir in sorted(os.listdir(root_dir), key=lambda x: int(x)):
        step = int(step_dir)
        grad_dir = os.path.join(root_dir, step_dir, 'grads')
        
        if os.path.isdir(grad_dir):
            for tensor_file in os.listdir(grad_dir):
                tensor_name = os.path.splitext(tensor_file)[0]
                tensor_path = os.path.join(grad_dir, tensor_file)
                
                # Load the tensor
                tensor = torch.load(tensor_path)
                
                # Calculate mean and standard deviation
                mean = tensor.mean().item()
                std = tensor.std().item()
                
                if tensor_name not in gradients:
                    gradients[tensor_name] = {'steps': [], 'means': [], 'stds': []}
                
                gradients[tensor_name]['steps'].append(step)
                gradients[tensor_name]['means'].append(mean)
                gradients[tensor_name]['stds'].append(std)

    # Plotting
    plt.figure(figsize=(12, 6))
    
    for tensor_name, data in gradients.items():
        steps = np.array(data['steps'])
        means = np.array(data['means'])
        stds = np.array(data['stds'])
        
        plt.plot(steps, means, label=tensor_name)
        plt.fill_between(steps, means - stds, means + stds, alpha=0.2)

    plt.xlabel('Training Step')
    plt.ylabel('Mean of Gradients')
    plt.title('Gradient Means and Variances Across Training Steps')
    plt.legend()
    plt.grid(True)
    plt.show()

# Usage
root_directory = '/fsx/phuc/projects/nanotron/debug/runs/exp33_debug_normalize_grad_of_balances_and_divide_by_10_and_lr_0.00015'

In [19]:
import os
import torch
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def load_and_plot_gradients(root_dir):
    gradients = {}

    # Walk through the directory structure
    for step_dir in sorted(os.listdir(root_dir), key=lambda x: int(x)):
        step = int(step_dir)
        grad_dir = os.path.join(root_dir, step_dir, 'grads')
        
        if os.path.isdir(grad_dir):
            for tensor_file in os.listdir(grad_dir):
                tensor_name = os.path.splitext(tensor_file)[0]
                tensor_path = os.path.join(grad_dir, tensor_file)
                
                # Load the tensor
                tensor = torch.load(tensor_path)
                
                # Calculate mean and standard deviation
                mean = tensor.mean().item()
                std = tensor.std().item()
                
                if tensor_name not in gradients:
                    gradients[tensor_name] = {'steps': [], 'means': [], 'stds': []}
                
                gradients[tensor_name]['steps'].append(step)
                gradients[tensor_name]['means'].append(mean)
                gradients[tensor_name]['stds'].append(std)

    # Create subplots
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

    # Add traces for each tensor
    for tensor_name, data in gradients.items():
        steps = data['steps']
        means = data['means']
        stds = data['stds']

        # Add line trace
        fig.add_trace(
            go.Scatter(
                x=steps,
                y=means,
                mode='lines',
                name=tensor_name,
                line=dict(width=2)
            )
        )

        # Add fill for standard deviation
        fig.add_trace(
            go.Scatter(
                x=steps + steps[::-1],
                y=[m + s for m, s in zip(means, stds)] + [m - s for m, s in zip(means[::-1], stds[::-1])],
                fill='toself',
                fillcolor='rgba(0,100,80,0.2)',
                line=dict(color='rgba(255,255,255,0)'),
                hoverinfo="skip",
                showlegend=False,
                name=tensor_name + ' Std Dev'
            )
        )

    # Update layout
    fig.update_layout(
        title='Gradient Means and Variances Across Training Steps',
        xaxis_title='Training Step',
        yaxis_title='Mean of Gradients',
        hovermode='x unified',
        legend_title='Tensor Names',
    )

    # Show the plot
    fig.show()

# Usage
root_directory = 'path/to/your/root/directory'
load_and_plot_gradients(root_directory)

ModuleNotFoundError: No module named 'plotly'

In [39]:
import os
import torch
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def load_and_plot_gradients(root_dir):
    gradients = {}

    # Walk through the directory structure
    for step_dir in sorted(os.listdir(root_dir), key=lambda x: int(x)):
        step = int(step_dir)
        grad_dir = os.path.join(root_dir, step_dir, 'grads')
        
        if os.path.isdir(grad_dir):
            for tensor_file in os.listdir(grad_dir):
                tensor_name = os.path.splitext(tensor_file)[0]
                tensor_path = os.path.join(grad_dir, tensor_file)
                
                # Load the tensor
                tensor = torch.load(tensor_path)
                
                # Calculate mean and standard deviation
                mean = tensor.mean().item()
                std = tensor.std().item()
                
                if tensor_name not in gradients:
                    gradients[tensor_name] = {'steps': [], 'means': [], 'stds': []}
                
                gradients[tensor_name]['steps'].append(step)
                gradients[tensor_name]['means'].append(mean)
                gradients[tensor_name]['stds'].append(std)

    # Create subplots
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

    # Add traces for each tensor
    for tensor_name, data in gradients.items():
        steps = data['steps']
        means = data['means']
        stds = data['stds']

        # Add line trace
        fig.add_trace(
            go.Scatter(
                x=steps,
                y=means,
                mode='lines',
                name=tensor_name,
                line=dict(width=2)
            )
        )

        # Add fill for standard deviation
        fig.add_trace(
            go.Scatter(
                x=steps + steps[::-1],
                y=[m + s for m, s in zip(means, stds)] + [m - s for m, s in zip(means[::-1], stds[::-1])],
                fill='toself',
                fillcolor='rgba(0,100,80,0.2)',
                line=dict(color='rgba(255,255,255,0)'),
                hoverinfo="skip",
                showlegend=False,
                name=tensor_name + ' Std Dev'
            )
        )

    # Update layout
    fig.update_layout(
        title='Gradient Means and Variances Across Training Steps',
        xaxis_title='Training Step',
        yaxis_title='Mean of Gradients',
        hovermode='x unified',
        legend_title='Tensor Names',
    )

    # Show the plot
    # fig.show()
    return fig, gradients

# Usage
root_directory = '/fsx/phuc/projects/nanotron/debug/runs/exp33_debug_normalize_grad_of_balances_and_divide_by_10_and_lr_0.00015'
fig, gradients = load_and_plot_gradients(root_directory)

In [40]:
fig.show()

In [35]:
from plotly.offline import init_notebook_mode, iplot


In [41]:
iplot(fig)

In [37]:
init_notebook_mode(connected=True)


In [4]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import numpy as np

# Initialize plotly for use in the notebook
init_notebook_mode(connected=True)

# Generate some sample data
x = np.linspace(0, 10, 100)
y1 = np.sin(x)
y2 = np.cos(x)

# Create traces
trace1 = go.Scatter(
    x = x,
    y = y1,
    mode = 'lines',
    name = 'sin(x)'
)
trace2 = go.Scatter(
    x = x,
    y = y2,
    mode = 'lines',
    name = 'cos(x)'
)

# Create layout
layout = go.Layout(
    title = 'Simple Interactive Plotly Graph',
    xaxis = dict(title = 'X-axis'),
    yaxis = dict(title = 'Y-axis')
)

# Combine trace and layout
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Plot!

### Exp 32

In [7]:
exp32_weights = read_tensor("/fsx/phuc/new_workspace/experiments/infini_attention_8b_llama/exp32_200m_infini_llama2_256_ctx_length_and_64_segment_length_and_2m_bs/checkpoints/10000/model/model/decoder/2/pp_block/attn/model_balance_factors_pp-rank-0-of-1_tp-rank-0-of-2.safetensors")

In [8]:
F.sigmoid(exp32_weights)

tensor([0.4961, 0.4922, 0.4941, 0.4922], dtype=torch.bfloat16)

In [9]:
F.sigmoid(read_tensor("/fsx/phuc/new_workspace/experiments/infini_attention_8b_llama/exp32_200m_infini_llama2_256_ctx_length_and_64_segment_length_and_2m_bs/checkpoints/10000/model/model/decoder/2/pp_block/attn/model_balance_factors_pp-rank-0-of-1_tp-rank-1-of-2.safetensors"))

tensor([0.4961, 0.4922, 0.4941, 0.4922], dtype=torch.bfloat16)

In [10]:
F.sigmoid(read_tensor("/fsx/phuc/new_workspace/experiments/infini_attention_8b_llama/exp32_200m_infini_llama2_256_ctx_length_and_64_segment_length_and_2m_bs/checkpoints/10000/model/model/decoder/0/pp_block/attn/model_balance_factors_pp-rank-0-of-1_tp-rank-1-of-2.safetensors"))

tensor([0.4922, 0.4922, 0.4922, 0.4961], dtype=torch.bfloat16)

In [11]:
F.sigmoid(read_tensor("/fsx/phuc/new_workspace/experiments/infini_attention_8b_llama/exp32_200m_infini_llama2_256_ctx_length_and_64_segment_length_and_2m_bs/checkpoints/10000/model/model/decoder/0/pp_block/attn/model_balance_factors_pp-rank-0-of-1_tp-rank-0-of-2.safetensors"))

tensor([0.4922, 0.4922, 0.4922, 0.4961], dtype=torch.bfloat16)